# ECE 4300 Not Hotdog

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import skimage.io
import skimage.transform
import urllib
import requests
from io import BytesIO
import flickrapi

In [ ]:
api_key = '8fa193368446fcb9fbedaea28db13ccc'
api_secret = 'e032c5956a398644'

flickr = flickrapi.FlickrAPI(api_key, api_secret)

# Pull images from Flickr

In [ ]:
import os
import warnings
from random import seed
from random import random

directory = 'hotdogs'
keyword = 'hotdog hotdogs food'
photos = flickr.walk(text=keyword, extras='url_c', sort='relevance', per_page=100)

# declare directory names
train_dir = 'train'
test_dir = 'test'
image_dir = directory

# make a directories for the training and testing images
for root_dir in [train_dir,test_dir]:
    dir_path = root_dir + '/' + image_dir

    # check if data exists
    if not os.path.isdir(root_dir):
        os.mkdir(root_dir)
    if not os.path.isdir(dir_path):
        os.mkdir(dir_path)
        print("Making directory %s" % dir_path)
    else:
        print("Will store %s images in directory %s" %(root_dir,dir_path))
    
ntrain = 10 # number of training images to download
ntest = 10 # number of testing images to download

nrow = 224 # image dimension 1
ncol = 224 # image dimension 2

# download images, first train then test
seed(1) # make repeatable
itrain = 0 # initialize counter
itest = 0 # initialize counter
for photo in photos:
    url=photo.get('url_c')
    if not (url is None):
        
        # Create a file from the URL
        # This may only work in Python3
        response = requests.get(url)
        file = BytesIO(response.content)
        
        # Read image from file
        im = skimage.io.imread(file)
        
        # Resize images
        im1 = skimage.transform.resize(im,(nrow,ncol),mode='constant')
                
        # Convert to uint8, suppress the warning about the precision loss
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            im2 = skimage.img_as_ubyte(im1)
    
        # Set the directory randomly 
        #if (random()< ntrain/(ntrain+ntest)):
        if (random()< (ntrain-itrain)/(ntrain-itrain + ntest-itest + 1e-15)):
            if (itrain<ntrain):
                dir_path = train_dir + '/' + image_dir
                i = itrain
                itrain = itrain + 1
            elif (itest<ntest):
                dir_path = test_dir + '/' + image_dir
                i = itest
                itest = itest + 1
            else:
                break
        else:
            if (itest<ntest):
                dir_path = test_dir + '/' + image_dir
                i = itest
                itest = itest + 1
            elif (itrain<ntrain):
                dir_path = train_dir + '/' + image_dir
                i = itrain
                itrain = itrain + 1
            else:
                break            
    
        # Save the image
        local_name = '{0:s}/{1:s}_{2:04d}.jpg'.format(dir_path,keyword,i)  
        skimage.io.imsave(local_name, im2)      
        print(local_name) 

In [ ]:
# Display the image
def disp_image(im):
    if (len(im.shape) == 2):
        # Gray scale image
        plt.imshow(im, cmap='gray')    
    else:
        # Color image.  
        im1 = (im-np.min(im))/(np.max(im)-np.min(im))*255
        im1 = im1.astype(np.uint8)
        plt.imshow(im1)    
        
    # Remove axis ticks
    plt.xticks([])
    plt.yticks([])

# Print some of the training images

In [ ]:
dir_path = train_dir + '/' + image_dir
plt.figure(figsize=(20,20))
nplot = 6
for i in range(nplot):
    fn = '{0:s}/{1:s}_{2:04d}.jpg'.format(dir_path, keyword, i)  
    im = skimage.io.imread(fn)
    plt.subplot(1,nplot,i+1)
    disp_image(im)

# Print some of the test images

In [ ]:
dir_path = test_dir + '/' + image_dir
plt.figure(figsize=(20,20))
nplot = 6
for i in range(nplot):
    fn = '{0:s}/{1:s}_{2:04d}.jpg'.format(dir_path,keyword, i)  
    im = skimage.io.imread(fn)
    plt.subplot(1,nplot,i+1)
    disp_image(im)